In [1]:
from Bio import SeqIO # Permet l'import de la fonction parse
from Bio.SeqIO.QualityIO import FastqGeneralIterator # Permet un parse plus rapide lorsque beaucoup de séquences (fastaq uniquement sinon voir site biopython pour fasta)
import functools as ft # Permet de regrouper des listes pour radix sort
import numpy as np

from tqdm import tqdm # Permet d'estimer le temps d'éxécution sur un boucle
from datetime import datetime # Permet de comparer la vitesse de 2 programmes 

### Import des listes reads et genome

In [2]:
def genome_import() :
    """
    Fonction qui importe depuis un fichier fasta la séquence du génome dans une liste contenant ["seq", "n°k"]
    """
    list_genom=[]
    for record in SeqIO.parse("GCF_000002765.5_GCA_000002765_genomic.fna","fasta"):
        list_genom.append([str(record.seq).upper(),record.description[-14:]]) # attention le marquage est à changer pour la dernière séquence
    
    return list_genom

def reads_import_cuts(k) :
    """
    Fonction qui importe depuis un fichier fasta les séquences des reads et les coupes en les rangeant
    dans une liste contenant ["seq", "nom", "n°kmer"]
    
    Entrée : k, int : longueur du kmer
    
    Sortie : list_reads : liste de tous les read découpés en kmers
    """
    list_reads=[]
    with open("single_Pfal_dat.fq") as in_handle:
        for title, seq, qual in tqdm(FastqGeneralIterator(in_handle), desc = "Import sequence"):
            i = 1    # Incrémenteur du nombre de k-mer
            while len(seq) >= 1 : # On parcoure toute la séquence
                if len(seq) > k : # Cas où le k-mer est entier
                    list_reads.append([str(seq[0:k]).upper(),title, i])
                    i += 1
                    seq = seq[k:]
                else : # Cas où le dernier k-mer n'est pas entier
                    list_reads.append([str(seq).upper(),title, i])
                    seq = ""
            
    return list_reads

In [3]:
now = datetime.now()

list_genom = genome_import()

list_reads= reads_import_cuts(10)

time_imp = datetime.now() - now
print("Temps total =", time_imp)
print(list_reads[0:6])

Import sequence: 1500000it [00:36, 41575.10it/s]

Temps total = 0:00:36.568160
[['TTTCCTTTTT', 'NC_004325.2-100000', 1], ['AAGCGTTTTA', 'NC_004325.2-100000', 2], ['TTTTTTAATA', 'NC_004325.2-100000', 3], ['AAAAAAATAT', 'NC_004325.2-100000', 4], ['AGTATTATAT', 'NC_004325.2-100000', 5], ['AGTAACGGGT', 'NC_004325.2-100000', 6]]


### Import de la DC3

In [4]:
def import_file(filename, prog) :
    
    if prog == "DC3" :
            filename = "DC3_save/" + filename
    elif prog == "Map" :
            filename = "Mapping_save/" + filename
    
    with open(filename, 'r') as f:
        
        # Partie DC3:
        if prog == "DC3" :
            text = f.readline()
            data = text.split(" ")[:-1]
            print(data)
            for i in range(len(data)) :
                data[i] = int(data[i])
            return(data)
        
        elif prog == "Map" :
            n = int(f.readline())
            data = []
            for i in range(n):
                temp = f.readline()
                temp = temp.split("&")
                temp_lis = temp[3][1:-2].split(",") # Car il reste les []
                lis = []
                for j in range(len(temp_lis)) :
                    lis.append(int(temp_lis[j]))
                data.append([temp[0], temp[1], int(temp[2]), lis])
            return data

In [5]:
DC3_chrom1 = import_file("DC3_chrom1.txt", "DC3")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Fonction de string search

In [6]:
def BWT(text,suffix_table):
    """
    Compute the BWT from the suffix table

    Args:
        T (str): string
        end_of_string (char): end of string character to append

    Return:
        bwt (str): BWT
    """
    bwt = ""
    sf_tab = suffix_table
    for i in range(len(sf_tab)):
        crt = sf_tab[i]
        bwt += text[crt-1]
    return(bwt)

In [51]:
def pattern_matching_BWT(S,pattern,bwt,index,somme):
    """
    Search a pattern in a String using the BWT

    Args:
        S (str): string
        pattern (str): pattern
        bwt : the bwt of the text (to not compute it each time)

    Return:
        bool: true if the pattern is in the string
        int : position of the first occurence of the pattern in the ordered text
        int : position of the last occurence of the pattern in the ordered text
    """
    pattern_in_S = False
    pot = []
    pat = []
    L = list(bwt)
    lpattern = list(pattern)
    start_string = -1
    end_string = -1
    ##init des valeurs utiles pour la substring search
    e = 0
    f = len(L)
    i = len(lpattern)-1
    ##début de la boucle de recherche
    while e < f and i > 0 :
        X = lpattern[i]
        Y = lpattern[i-1]
        r = 0
        s = 0
        suite_impos = True
        for tpl in somme :
            if tpl[0]<X:
                #print(tpl[0])
                r = tpl[1] ##donne place du premier char dans la liste ordonnée
            if tpl[0]==X:
                s = tpl[1]-1 ##donne place du dernier char
        if e>r:
            r = e
        if f<s:
            s = f
        for u in range(r,s+1):
            if(suite_impos==False):##we use the boolean to exit the loop early
                break
            if L[u]==Y:
                print(L[u], Y)
                suite_impos= False
                prev = 0
                idx = index[u]
                
                for tpl in somme :
                    if tpl[0]<Y:
                        prev = tpl[1]
                e = prev + idx[1] -1
                start_string = e
        char_found = False  ##this allows to exit the loop early if the char has been found
        for u in reversed(range(r,s+1)):
            if(char_found or suite_impos):##on sort de la boucle si on trouve le char, ou si on sait déja qu'il n'est pas la
                break
            if L[u]==Y:
                char_found = True
                prev = 0
                idx = index[u]
                
                for tpl in somme :
                    if tpl[0]<Y:
                        prev = tpl[1]
                f = prev + idx[1] -1
                end_string = f
        if suite_impos :## this will stop the loop if no char has been found in the previous one
            break
        
        i-=1

    if suite_impos:
        i = 0
        pattern_in_S = False
        print(start_string,end_string)
        return pattern_in_S,start_string,end_string
    ##dans le cas où e = f, il faut vérifier que le reste du substring est bon
    if i > 0 :
        while i > 0 :
            if L[e] != lpattern[i-1]:
                break
            prev = 0
            idx=index[e]
            start_string = e
            end_string = e
            for tpl in somme :
                if tpl[0]<L[e]:
                    prev = tpl[1]

            e = prev + idx[1]
            i -= 1
    if i < 1 :
        print(pot)
        print(pat)
        pattern_in_S = True

    return pattern_in_S,start_string,end_string

In [54]:
def string_location(text,string,matches,suffix_table):
    '''
    Gives the position of each occurence of the substring in the text

    Args :
        text (string) : the text to search in
        string (string) : the substring to be searched

    Return :
        ???
    '''
    result = matches
    print(result[1],result[2])
    sft = suffix_table
    list_occur = []
    if result[0] == False :
        print("No occurence of the substring was found")
        list_occur.append(-1)
    else :
        for i in range(result[1],(result[2]+1)):
            ##print(text[sft[i-1]-1],"ici")
            idx = sft[i]
            list_occur.append(idx)
            print(text[idx:idx+len(string)])
    return(list_occur)

def k_positioning(text,patt,bwt,suffix_table):##permet d'obtenir la liste des positions
    ##initialisation de l'alphabet, de l'index et du compteur total de char
    L = list(bwt)
    alphabet = ['$','A','C','G','T']
    total = []
    index = L+[]
    for lett in alphabet:
        cntr = 0
        for i in range(len(L)):
            if L[i]==lett:
                cntr +=1
                index[i]=(lett,cntr)
        total.append((lett,cntr))##le faire en une liste somme et total
    somme = []
    som=0
    for tpl in total:
        som += tpl[1]
        somme.append((tpl[0],som))
    ##recuperation des positions des premiers et derniers patterns trouvés
    mat = pattern_matching_BWT(text,patt,bwt,index,somme)
    ##recupération et renvoi des positions de tout les patterns
    return string_location(text,patt,mat,suffix_table)

### Appel de la fonction :

In [57]:
sf_tab = DC3_chrom1

text = list_genom[0][0] # Corespond au DC3_chrom1 non DC3
pattern = list_reads[0][0] #Liste des pattern donc 15000000 de possibilité, changer le premier terme pour passer au pattern suivant

bwt = BWT(text,sf_tab)

result  = k_positioning(text,pattern,bwt,sf_tab)
print(result, len(result))
print(sf_tab[608648:608704+1])

T T
T T
T T
T T
C C
C C
T T
T T
T T
[]
[]
608648 608704
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
TTTCCTTTTT
[198275, 346984, 412675, 222197, 222208, 222219, 141536, 278529, 472382, 95770, 370439, 308410, 373485, 593890, 68418, 593802, 423536, 536793, 249822, 201291, 380365, 569951, 361808, 434544, 52645, 389576, 293273, 73765, 444636, 412568, 253527, 436364, 436171, 436187, 436203, 436219, 436235, 436251, 436267, 436283, 

In [42]:
print(list_reads[0][0]) # Affiche le vrai pattern pour le moment majoritairement retrouvé mais quelques erreurs d'identifications

TTTCCTTTTT


In [11]:
print(list_genom[0][0][346984:346994]) # Indice du premier pattern : ok position correcte !

TTTCCTTTTT


In [12]:
T = "TTTCCAATTAATTATCAAGTCTGTTTTCCAATACTAGCTGCATCGATCGTAAGCATCAAGTCTGTTTTGGGTTTCCAATTAATTATCAAGTTTCCAATTAATTATCAAGTCTGTTTTGGGTTTCCAATTAATTATCAAGTCTGTTTTGGGACTCTGCATCTGTTTTGGGACTCTGCATTTGGGTTTCCAATTAATTATCAAGTCTGTTTTGGGACTCTGCA"
T2 = "TTTCCAATTAATTATCAAGTCTGTTTTGGGTTTCCAATTAATTATCACCAGTCGTATTTTGGGACTCTGCACCTAATCCCCAACACTTTGTCGTAGAAACACTTTGAG"
patt = "ATCAA"
patt5 = "AGTCGTA"
patt4 = "AGTCGT"
patt2 = "TGCACC"
text= T + '$'
sf_tab = DC3(text)
print(len(T))
sft=DC3(T)
bwt = BWT(text,sf_tab)
print(k_positioning(text,patt,bwt,sf_tab))

NameError: name 'DC3' is not defined

In [ ]:
def DC3(S, P_12_base = []) :
    """
    Create the suffix array from DC3 algorithm, could be recursive if needed
    
    Args:
        S (str): string
        P_12_base : store P1+2 from recursion to map correctly recursivity
    
    Return:
        index_012 : suffix array of S
        order_12 : order of the next recursion to map correctly recursivity
    """
    
    DC3_table = np.zeros((3,len(S) + 3), dtype=int) # On crée un tableau de taille longeur de la séquence + 3 caractères sentinelles
    """
    Table de DC3 qui contient en chaque ligne : 
    Ligne 0 : indice du caractère ( ducoup optionel si on veut )
    Ligne 1 : conversion du caractère en nombre
    Ligne 2 : Ordre de l'indice du caractère
    
    """
    for i in range(len(S) + 3) :
            DC3_table[0][i] =  i # On initialise les indices
    
    #print(DC3_table)
    
    
    # String conversion : !!!!! à n'executer que lors de la première récursion !!!!!
    if type(S) == str :
        S_l = [*S] # On sépare caractère par caractère : "ATGC" devient ["A","T","G","C"]
        for i in range(len(S_l)) :
            DC3_table[1][i] =  ord(S_l[i]) # On rempli le caractère par son code Ascii dans la table
    else :
         for i in range(len(S)) :
            DC3_table[1][i] =  S[i] # Cas où l'on rentre dans la boucle une deuxième fois ou plus, pas de conversion
    
    # Cas où la chaine n'est composé que d'une seule lettre : trivial car DC3 = ordre des indices en décroissant
    equal = True
    for i in range(len(S)) :
        if DC3_table[1][0] != DC3_table[1][i] :
            equal = False
            break # On teste si la chaine n'est composé que d'une seule lettre
    if equal == True :
         return [*range(len(S)-1,-1,-1)]
    
    
    """
    for i in range(len(S), len(S) + 3, 1) :
        conversion_l.append(0) # Ajout des 3 caractères sentinelles # A garder si pas tables de 0
    """
    
    # On crée P0, P1, P2 et P1+P2 :
    
    P0 = [*range(0,len(S)+1,3)] 
    P1 = [*range(1,len(S)+1,3)]
    P2 = [*range(2,len(S)+1,3)]
    
    P_12 = P1 + P2
    #print(P0, P1, P2)
    #print(P_12)
    
    #Obtention des triplets à partir de P1+P2 :
    
    R_12 = []
    for val in P_12 :
        R_12.append([list(DC3_table[1][val:val+3]), val])

    print(R_12)
    
    radix_sort(R_12,3) # On trie les triplets
    
    print(R_12)
    
    index_12 = [] # Liste des indexes de R12 trié
    order_count = 1 # Compteur pour remplir l'ordre
    recur = False # Etat de la récursion tourné True si on a des égalités d'ordre
    for j in range(len(R_12)) : # On parcours tous les triplets triés
        index_12.append(R_12[j][1]) # ... pour lui attribuer son index depuis P_12
        DC3_table[2][R_12[j][1]] = order_count # Et on ajoute l'ordre dans la table
        if j < len(R_12)-1 :
            if R_12[j][0] != R_12[j+1][0] : # On teste l'égalité des triplets pour mettre l'ordre
                order_count += 1
            else :
                recur = True # On a égalité, donc on doit relancer l'algorithme à la fin des for
        else :
            order_count += 1
    
    #print(DC3_table, index_12)
    
    if recur == True :
        new_S = [] # On crée T' la séquence des orders suivant l'ordre de P12
        for l in P_12 :
            new_S.append(DC3_table[2][l])
        #print(new_S)
        index_012 = DC3(new_S, P_12) # On doit récupérer ces deux paramètres sinon ça marche pas
        index_12 = []
        for ind,val in index_012 :
            DC3_table[2][ind] = val
            index_12.append(ind)
            
        print(DC3_table)
    
    R_0 = [] # On crée la dernière partie à trier
    for val in P0 :
        R_0.append([[int(DC3_table[1][val]), DC3_table[2][val + 1]], val]) # On crée R0 avec son indice
        
    print(R_0)
    
    radix_sort(R_0,2)
    
    print(R_0)
    
    index_0 = [] # Liste des indexes de R0 trié
    for k in range(len(R_0)) : # On parcours tous les doublets triés
       index_0.append(R_0[k][1]) # On récupère l'indice
    
    #print(DC3_table[0:2], index_0, index_12)
    
    index_012 = [] # On crée l'index final en ordonant 0 et 1,2
    i_0 = 0
    i_12 = 0
    
    index_12_dict = {}
    for i in range(len(index_12)) :
        index_12_dict[index_12[i]] = i
        
    while (i_0 < len(index_0) and i_12 < len(index_12)) : # On prends tout les éléments : on vide index 0 ou 12
        val_i0 = index_0[i_0]
        val_i12 = index_12[i_12]
        
        if DC3_table[1][val_i0] > DC3_table[1][val_i12] : # Cas où index 12 arrive avant index 0
            index_012.append(index_12[i_12])
            i_12 += 1
        elif DC3_table[1][val_i0] < DC3_table[1][val_i12] : # Cas où index 0 arrive avant index 12
            index_012.append(index_0[i_0])
            i_0 += 1
        else : # Cas d'égalité sur l'indice : si les 2 indexes renvoient le même nombre 
            if index_12[i_12] % 3 == 1 :
                if index_12_dict[val_i0 + 1] > index_12_dict[val_i12 + 1] : # Cas où index 12 au deuxième terme arrive avant index 0 au deuxième terme
                    index_012.append(index_12[i_12])
                    i_12 += 1
                else : # Cas où index 0 au deuxième terme arrive avant index 12 au deuxième terme
                    index_012.append(index_0[i_0])
                    i_0 += 1
            else :
                if DC3_table[1][val_i0 + 1] > DC3_table[1][val_i12 + 1] : # On teste dabord cas où index 12 + 1 arrive avant index 0 + 1
                    index_012.append(index_12[i_12])
                    i_12 += 1
                elif DC3_table[1][val_i0 + 1] < DC3_table[1][val_i12 + 1] : # Cas où index 0 + 1 arrive avant index 12 + 1
                    index_012.append(index_0[i_0])
                    i_0 += 1
                elif index_12_dict[val_i0 + 2] > index_12_dict[val_i12 + 2] : # Cas où index 12 au deuxième terme arrive avant index 0 au troisième terme
                    index_012.append(index_12[i_12])
                    i_12 += 1
                else : # Cas où index 0 au troisième terme arrive avant index 12 au deuxième terme
                    index_012.append(index_0[i_0])
                    i_0 += 1
    
    index_012.extend(index_12[i_12:]) # Si 1 des deux index est encore plein, on ajoute son contenu
    index_012.extend(index_0[i_0:])
    """
    while (i_0 < len(index_0) or i_12 < len(index_12)) : # On prends tout les éléments : on vide index 0 et 12
        if i_0 == len(index_0) : # Cas où index 0 est vide, on rempli avec index 12
            index_012.extend(index_12[i_12:])
            i_12 += len(index_12)
        elif i_12 == len(index_12) : # Cas où index 12 est vide, on rempli avec index 0
            index_012.extend(index_0[i_0:])
            i_0 += len(index_0)
        elif DC3_table[1][index_0[i_0]] > DC3_table[1][index_12[i_12]] : # Cas où index 12 arrive avant index 0
            index_012.append(index_12[i_12])
            i_12 += 1
        elif DC3_table[1][index_0[i_0]] < DC3_table[1][index_12[i_12]] : # Cas où index 0 arrive avant index 12
            index_012.append(index_0[i_0])
            i_0 += 1
        else : # Cas d'égalité sur l'indice : si les 2 indexes renvoient le même nombre 
            if index_12[i_12] % 3 == 1 :
                if index_12.index(index_0[i_0] + 1) > index_12.index(index_12[i_12] + 1) : # Cas où index 12 au deuxième terme arrive avant index 0 au deuxième terme
                    index_012.append(index_12[i_12])
                    i_12 += 1
                else : # Cas où index 0 au deuxième terme arrive avant index 12 au deuxième terme
                    index_012.append(index_0[i_0])
                    i_0 += 1
            else :
                if DC3_table[1][index_0[i_0] + 1] > DC3_table[1][index_12[i_12] + 1] : # On teste dabord cas où index 12 + 1 arrive avant index 0 + 1
                    index_012.append(index_12[i_12])
                    i_12 += 1
                elif DC3_table[1][index_0[i_0] + 1] < DC3_table[1][index_12[i_12] + 1] : # Cas où index 0 + 1 arrive avant index 12 + 1
                    index_012.append(index_0[i_0])
                    i_0 += 1
                elif index_12.index(index_0[i_0] + 2) > index_12.index(index_12[i_12] + 2) : # Cas où index 12 au deuxième terme arrive avant index 0 au troisième terme
                    
                    index_012.append(index_12[i_12])
                    i_12 += 1
                else : # Cas où index 0 au troisième terme arrive avant index 12 au deuxième terme
                    index_012.append(index_0[i_0])
                    i_0 += 1
    #print(index_012)
    """
    
    if int(DC3_table[1][index_012[0]]) == 0 : # On enlève le terme sentinel s'il est présent
        index_012 = index_012[1:]
        
    #print(index_012)
    
    if len(P_12_base) > 0 : # Mapping sur recursion -1 si existe
        new_index_012 = []
        for n in range(len(index_012)) :
            new_index_012.append([P_12_base[index_012[n]], n])
        #print(new_index_012)
        index_012 = new_index_012
   
    

    return index_012 # Retourne le suffix array si dernière récursion

In [ ]:
def flatten(arr):
    # voir import
    return ft.reduce(lambda x, y: x + y, arr)

def counting_sort(array,digit,p): 
    """
    Nécessaire pour le radix sort, je te le laisse Erwan
    """
    ##The counting sort is a stable sort used in the radic sort
    ##here the counting sort needs to be adapted to look at only one digit of each number (for radix)
    ##we also added the parameter p to be able to read the triplets (more info below)
    ##1) since we only sort single digits, the max will always be smaller than 10
    ##2) create count index (that will have the cumulative index in the end)
    count_list = [[]for i in range(10)]
    for tpl in array :
        n_uplet = tpl[0]##we take the first element of the tuple, the n-uplet
        num = n_uplet[p] // digit ##we cut off the digits to the right by getting the quotient of the euclidian division
        ##p is the index of the number studied in the triplet
        count_list[num%10].append(tpl)##we cut off the digits to the left by getting the remainder of the euclidian division
    arr_ord = flatten(count_list)

    for i in range(len(array)):##we change the base array to allow the radix sort to loop easily
        array[i] = arr_ord[i]
        
def radix_sort(array,p):
    """
    Prend en entrée un tableau de valeur de valeur et le trie en foncion de la première composante
    selon l'algorithm du radix sort
    
    Entrée : array à trier
             p : nombre de cases à trier p-uplets
    
    Sortie : array trié
    """
    ##Here the radix sort is modified to work with the triplet list sent by the DC3
    ##The code is not flexible enough to compute all characters in the ascii table, but it's enough for the use needed
    ##1) we search for the max in the nuplets
    mx = (max(array)[0])[0]+1
    if p != 3 : ##we take max = 100 because A T C G are all below 100 in ascii code
        for tpl in array:
            n_uplet = tpl[0]
            if n_uplet[-1] > mx :
                mx = n_uplet[-1]+1
    '''##2) to know how many loops we have to do, we will use a variable to represent,
    the digit we are currently in'''
    for i in reversed(range(0,p)):
        digit = 1 ##starts at one for units
        while mx - digit > 0 :##when all the digits are checked, digit will be greater than the maximum
            counting_sort(array,digit,i)
            digit *= 10 ##digit will go to the tens, the hundreds, the thousands...